In [ ]:
import pickle
pickle.HIGHEST_PROTOCOL = 4

import numpy as np
import pandas as pd

import os
from pathlib import Path
import func_proc_filepath as mFILE
from tqdm import tqdm
from IPython.display import display
import shutil

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

import tensorflow as tf
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.optimizers import Adam
from keras import backend as K

import random

from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint

import warnings
warnings.filterwarnings('ignore')

In [ ]:
class F1Callback(Callback):
    def __init__(self, model, X_val, y_val, path):
        self.model = model
        self.X_val = X_val
        self.y_val = y_val
        
        self.path = path
        self.best_score = 0

    def on_epoch_end(self, epoch, logs):
        pred = self.model.predict(self.X_val)
        f1_val = f1_score(self.y_val, np.round(pred), average='macro')
        acc_val = accuracy_score(self.y_val, np.round(pred))
        score = f1_val*0.67 + acc_val*0.33
        log = "Score = {0}, F1 = {1}, ACC = {2}".format(score, f1_val, acc_val)
        print(log)
        # 以下チェックポイントなど必要なら書く
        if score > self.best_score:
            self.best_score = score
            self.model.save(self.path)

In [ ]:
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
def df_expw_with_balance(df_data):
    #df_data = pd.read_hdf(fp)
    df_data.dropna(how='any', inplace=True)
    #display(df_data)
    
    df_0 = df_data.loc[df_data.loc[:,"Neutral"]==1,:]
    df_1 = df_data.loc[df_data.loc[:,"Anger"]==1,:]
    df_2 = df_data.loc[df_data.loc[:,"Disgust"]==1,:]
    df_3 = df_data.loc[df_data.loc[:,"Fear"]==1,:]
    df_4 = df_data.loc[df_data.loc[:,"Happiness"]==1,:]
    df_5 = df_data.loc[df_data.loc[:,"Sadness"]==1,:]
    df_6 = df_data.loc[df_data.loc[:,"Surprise"]==1,:]

    df_0 = df_0[::2]
    df_1 = pd.concat([df_1,df_1,df_1],axis=0)
    df_2 = pd.concat([df_2,df_2,df_2],axis=0)
    df_3 = pd.concat([df_3,df_3,df_3,df_3,df_3,df_3,df_3,df_3,df_3,df_3,df_3],axis=0)
    df_4 = df_4[::2]
    df_6 = pd.concat([df_6,df_6],axis=0)
    df_m = pd.concat([df_0, df_1, df_2, df_3, df_4, df_5, df_6],axis=0)
    
    return df_m

In [ ]:
def df_aff2_with_balance(df_data):
    df_data.dropna(how='any', inplace=True)
    #display(df_data)  
    
    df_0 = df_data.loc[df_data.loc[:,"expr"]==0,:]
    df_1 = df_data.loc[df_data.loc[:,"expr"]==1,:]
    df_2 = df_data.loc[df_data.loc[:,"expr"]==2,:]
    df_3 = df_data.loc[df_data.loc[:,"expr"]==3,:]
    df_4 = df_data.loc[df_data.loc[:,"expr"]==4,:]
    df_5 = df_data.loc[df_data.loc[:,"expr"]==5,:]
    df_6 = df_data.loc[df_data.loc[:,"expr"]==6,:]

    df_0 = df_0[::2]
    df_1 = pd.concat([df_1,df_1],axis=0)
    df_2 = pd.concat([df_2,df_2,df_2,df_2],axis=0)
    df_3 = pd.concat([df_3,df_3,df_3,df_3],axis=0)
    df_4 = df_4[::3]
    df_5 = df_5[::2]
    df_m = pd.concat([df_0, df_1, df_2, df_3, df_4, df_5, df_6],axis=0)
    
    return df_m

In [ ]:
def df_val_with_balance(df_data):
    df_data.dropna(how='any', inplace=True)
    #display(df_data)

    df_0 = df_data.loc[df_data.loc[:,"expr"]==0,:]
    df_1 = df_data.loc[df_data.loc[:,"expr"]==1,:]
    df_2 = df_data.loc[df_data.loc[:,"expr"]==2,:]
    df_3 = df_data.loc[df_data.loc[:,"expr"]==3,:]
    df_4 = df_data.loc[df_data.loc[:,"expr"]==4,:]
    df_5 = df_data.loc[df_data.loc[:,"expr"]==5,:]
    df_6 = df_data.loc[df_data.loc[:,"expr"]==6,:]

    df_0 = df_0[::3]
    #df_1 = pd.concat([df_1,df_1],axis=0)
    df_2 = pd.concat([df_2,df_2],axis=0)
    #df_3 = pd.concat([df_3,df_3,df_3,df_3],axis=0)
    df_4 = df_4[::4]
    df_5 = df_5[::3]
    df_6 = df_5[::2]
    df_m = pd.concat([df_0, df_1, df_2, df_3, df_4, df_5, df_6],axis=0)
    
    return df_m

In [ ]:
def create_pseudo_label(df_data, pre_model):
    
    df_data.dropna(how='any', inplace=True)
    np_x = df_data.loc[:, df_data.columns.str.contains("AU|pose_R|gaze|vgg-")].values
    pred_ = pre_model.predict(np_x)
    pred_p = np.argmax(pred_, axis=1)

    df_data["expr"] = pred_p
    df_0 = df_data.loc[((df_data.loc[:,"valence"]**2)<0.25)&((df_data.loc[:,"arousal"]**2)<0.25)&(df_data.loc[:,"expr"]==0),:]
    df_1 = df_data.loc[(df_data.loc[:,"valence"]<0)&(df_data.loc[:,"arousal"]>0)&(df_data.loc[:,"expr"]==1),:]
    df_2 = df_data.loc[(df_data.loc[:,"valence"]<0)&(df_data.loc[:,"arousal"]>0)&(df_data.loc[:,"expr"]==2),:]
    df_3 = df_data.loc[(df_data.loc[:,"valence"]<0)&(df_data.loc[:,"arousal"]>0)&(df_data.loc[:,"expr"]==3),:]
    df_4 = df_data.loc[(df_data.loc[:,"valence"]>0)&(df_data.loc[:,"arousal"]>0)&(df_data.loc[:,"expr"]==4),:]
    df_5 = df_data.loc[(df_data.loc[:,"valence"]<0)&(df_data.loc[:,"arousal"]<0)&(df_data.loc[:,"expr"]==5),:]
    df_6 = df_data.loc[(df_data.loc[:,"arousal"]>0)&(df_data.loc[:,"expr"]==6),:]
    
    df_data = None
    
    """
    ・0: V<0.5, A<0.5
　　・1: V<0,   A>0
　　・2: V<0,   A>0
　　・3: V<0,   A>0
　　・4: V>0,   A>0
　　・5: V<0,   A<0
　　・6:        A>0
    """
    
    print("balance", len(df_0), len(df_1), len(df_2), len(df_3), len(df_4), len(df_5), len(df_6))
    # 212377 10569 14561 1531 213667 210 67275
    df_0 = df_0[::10]
    #df_1 = df_1[::1]
    #df_2 = df_2[::1]
    df_3 = pd.concat([df_3,df_3,df_3,df_3,df_3],axis=0)
    df_4 = df_4[::20]
    df_5 = pd.concat([df_5,df_5,df_5,df_5,df_5,df_5,df_5,df_5,df_5,df_5,
                      df_5,df_5,df_5,df_5,df_5,df_5,df_5,df_5,df_5,df_5],axis=0)
    df_6 = df_6[::5]
    
    df_out = pd.concat([df_0, df_1, df_2, df_3, df_4, df_5, df_6], axis=0)
    df_out.drop(["valence", "arousal"], axis=1, inplace=True)

    return df_out


In [ ]:
# y: after one hot encoding
def create_model(np_train_x, np_train_y, np_val_x, np_val_y, n_class, drop=0.5, lr=1e-4, batch=128, 
                 model_path=""):
    
    # ** fix random seed **
    FIX_SEED = 49
    os.environ['PYTHONHASHSEED'] = '0'
    np.random.seed(FIX_SEED)
    random.seed(FIX_SEED)
    session_conf = tf.compat.v1.ConfigProto()
    tf.compat.v1.set_random_seed(FIX_SEED)
    sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
    tf.compat.v1.keras.backend.set_session(sess)
    # *********************
    
    #*********************************************    
    # モデルの定義
    model = Sequential()
    
    model.add(Dense(300, name='vgg_au')) #
    #model.add(BatchNormalization())
    #model.add(Activation('relu'))
    model.add(Activation('tanh'))
    model.add(Dropout(drop))
    model.add(Dense(n_class, activation='softmax'))
    adam = Adam(lr=lr)
    model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=["accuracy", f1])
    #model.compile(optimizer="Adam", loss='categorical_crossentropy', metrics=["accuracy"])

    #model_path = dir_out + "\\model_single_image_pseudo" + footer + ".h5"
    #cb_early = EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='auto')
    cb_early = EarlyStopping(monitor='val_f1', patience=8, verbose=0, mode='max')
    
    model.fit(np_train_x, np_train_y,
              epochs=30,
              batch_size=batch,
              validation_data=(np_val_x, np_val_y),
              callbacks=[F1Callback(model, np_val_x, np_val_y, model_path), cb_early])
    
    #score = model.evaluate(np_val_x, np_val_y_hot, batch_size=128)
    #display(score)
    model.summary()
    
    # bestモデルを読み込んで、重みやオプティマイザーを含むモデル全体を再作成
    new_model = keras.models.load_model(model_path, custom_objects={'f1':f1})
    
    pred_nn = new_model.predict(np_val_x)
    score_nn_f = f1_score(np.argmax(np_val_y, axis=1), np.argmax(pred_nn, axis=1), average='macro')
    score_nn_a = accuracy_score(np.argmax(np_val_y, axis=1), np.argmax(pred_nn, axis=1))
    score_nn = score_nn_f*0.67 + score_nn_a*0.33
    
    scores_nn = [score_nn, score_nn_f, score_nn_a]
    
    display(["0.67*F1+0.33*ACC", "F1 score", "ACC score"])
    display(scores_nn)
    
    del model
    del new_model
    
    return scores_nn

In [ ]:
# evaluate predict data
def eval_pred_each_class(np_true, np_pred, num_class):

    result = []

    # calc
    for i in range(num_class):
        # i以外を0、iを1に置き換える
        y_true_i = np.where(np_true == i, 1, 0)
        y_pred_i = np.where(np_pred == i, 1, 0)

        recall_i = recall_score(y_true_i, y_pred_i, average='binary')
        precision_i = precision_score(y_true_i, y_pred_i, average='binary')
        f1_i = f1_score(y_true_i, y_pred_i, average='binary')
        acc_i = accuracy_score(y_true_i, y_pred_i)

        df_reslut = pd.DataFrame({"class":[i], "recall":[recall_i], "precision":[precision_i], 
                                 "f1":[f1_i], "accuracy":[acc_i]})
        #result_i = [recall_i, precision_i, f1_i, acc_i]

        result.append(df_reslut)

    df_out = pd.concat([x for x in result], axis=0, ignore_index=True)
    f1_all = df_out.loc[:, "f1"].mean()
    recall_all = df_out.loc[:, "recall"].mean()
    precision_all = df_out.loc[:, "precision"].mean()
    accuracy_all = df_out.loc[:, "accuracy"].mean()
    df_all = pd.DataFrame({"class":["all"], "recall":[recall_all], "precision":[precision_all], 
                                 "f1":[f1_all], "accuracy":[accuracy_all]})
    df_out = pd.concat([df_out, df_all], axis=0, ignore_index=True)

    return df_out


In [ ]:
def main():
    # root folder
    dir_root = str(Path(Path().resolve()).parent)

    # input: folder path including original images
    dir_data = dir_root + "\\dataset\\aff2_images\\dataset"    

    # input: expr labels
    dir_label_train = dir_root + "\\src\\annotations\\EXPR_Set\\Train_Set"
    dir_label_val = dir_root + "\\src\\annotations\\EXPR_Set\\Validation_Set"
    
    file_data_expw = dir_root + "\\dataset\\expw_images\\dataset\\expw_with_label.h5"
    
    # output: folder path
    dir_out = dir_root + "\\model_expr\\model_image"
    if os.path.isdir(dir_out) == False:
        os.makedirs(dir_out)
        
    model_path_base = "model_image_single_pseudo"
    model_path_base_pre = "model_image_single"
        
    # for pseudo label ***************
    file_valid_va = dir_root + "\\parameters\\vaid_va_list.txt"
    dir_valid_va = dir_root + "\\parameters\\va_label"
    file_va_list = pd.read_csv(file_valid_va).loc[:, "file"].values.ravel()
    file_pre_model = dir_out + "\\" + model_path_base_pre + "_best.h5"
    pre_model = keras.models.load_model(file_pre_model, custom_objects={'f1':f1})
    
    log = "create pseudo label"
    print(log)
    df_pseudo_list = []
    for i in tqdm(range(len(file_va_list))):
        fp_label = dir_valid_va + "\\" + file_va_list[i]
        df_va_label = pd.read_csv(fp_label)
        df_va_label["frame"] = df_va_label.index
        name = os.path.splitext(os.path.basename(file_va_list[i]))[0]
        fp_va_data = dir_data + "\\" + name + ".h5"
        df_va_data = pd.read_hdf(fp_va_data)
        df_va_merge = pd.merge(df_va_label, df_va_data, on="frame")
        df_pseudo_list.append(df_va_merge)
    df_va_merge = pd.concat([x for x in df_pseudo_list], axis=0)
    df_va = create_pseudo_label(df_va_merge, pre_model)
    df_pseudo_list = None
    df_va_merge = None
    log = "pseudo label data: {0}".format(df_va.shape)
    print(log)
    # *****
    
    train_list = mFILE.search_files(dir_label_train, valid_names=[".txt"], invalid_names=["wuert"], ext=None, recursive=False)    
    val_list = mFILE.search_files(dir_label_val, valid_names=[".txt"], invalid_names=["wuert"], ext=None, recursive=False)
    
    len_train = len(train_list)
    len_val = len(val_list)
    #len_train = 75
    #len_val = 25
    
    df_train_list = []
    for i in tqdm(range(len_train)):
        name = os.path.splitext(os.path.basename(train_list[i]))[0]
        
        df_label = pd.read_csv(train_list[i])
        #display(df_label)
        df_label = df_label.drop(["Anger","Disgust","Fear","Happiness","Sadness","Surprise"],axis=1)
        df_label.columns = ["expr"]
        df_label["frame"] = df_label.index
        
        df_data = pd.read_hdf(dir_data + "\\" + name + ".h5")
        
        df_merge = pd.merge(df_label, df_data, on="frame")
        df_merge = df_merge.loc[df_merge.loc[:,"expr"]>=0,:]
        
        df_train_list.append(df_merge)

    df_train = pd.concat([x for x in df_train_list],axis=0)
    df_train_list = None #clear memory ***************
    df_train = df_aff2_with_balance(df_train)
    
    df_expr = pd.read_hdf(file_data_expw)
    df_expr = df_expw_with_balance(df_expr)
    
    #df_train = pd.concat([df_train, df_expr], axis=0)
    df_train = pd.concat([df_train, df_expr, df_va], axis=0)
    df_train = df_train.sample(frac=1, random_state=49)
    
    df_val_list = []
    for i in tqdm(range(len_val)):
        name = os.path.splitext(os.path.basename(val_list[i]))[0]
        
        df_label = pd.read_csv(val_list[i])
        df_label = df_label.drop(["Anger","Disgust","Fear","Happiness","Sadness","Surprise"],axis=1)
        df_label.columns = ["expr"]
        df_label["frame"] = df_label.index
        
        df_data = pd.read_hdf(dir_data + "\\" + name + ".h5")
        
        df_merge = pd.merge(df_label, df_data, on="frame")
        df_merge = df_merge.loc[df_merge.loc[:,"expr"]>=0,:]
        
        df_val_list.append(df_merge)
    
    df_val = pd.concat([x for x in df_val_list],axis=0)
    df_val = df_val_with_balance(df_val)
    df_val = df_val.sample(frac=1, random_state=49)
        
    #df_train.dropna(how='any', inplace=True)
    #df_val.dropna(how='any', inplace=True)
    df_val.fillna(0, inplace=True)
    #display(df_train.shape)
    #display(df_val.shape)
    display(df_train)
    display(df_val)
    
    np_train_x = df_train.loc[:, df_train.columns.str.contains("AU|pose_R|gaze|vgg-")].values
    np_train_y = df_train.loc[:, df_train.columns.str.contains("expr")].astype("int").values #.ravel()
    np_val_x = df_val.loc[:, df_val.columns.str.contains("AU|pose_R|gaze|vgg-")].values
    np_val_y = df_val.loc[:, df_val.columns.str.contains("expr")].astype("int").values #.ravel()
    
    display(np_train_x.shape, np_train_y.shape)
    display(np_val_x.shape, np_val_y.shape)
       
    df_train = None #clear memory ***************
    df_val = None #clear memory ***************
        
    dummy = [[0],[1],[2],[3],[4],[5],[6]]
    transformer = OneHotEncoder().fit(dummy)
    np_train_y_hot = transformer.transform(np_train_y).toarray()
    np_val_y_hot = transformer.transform(np_val_y).toarray()

    #\src\annotations\EXPR_Set\Train_Set
    
    len_feat = len(np_train_x[0])
    len_class = len(np_train_y_hot[0])
    display(len_feat)
    display(len_class)
    
    
    # search parameter
    score_list = []
    #***
    #l_units = [64, 128]
    l_drop = [0.3, 0.5]
    l_lr = [1e-4, 1e-3]
    l_batch = [128, 256]
    MAX_COUNT = len(l_drop)*len(l_lr)*len(l_batch)
    COUNT = 0
    
    for _drop in l_drop:
        for _lr in l_lr:
            for _batch in l_batch:
                model_path = dir_out + "\\" + model_path_base + "_{0:02d}.h5".format(COUNT)
                scores = create_model(np_train_x, np_train_y_hot, np_val_x, np_val_y_hot, len_class,
                                      drop=_drop, lr=_lr, batch=_batch, model_path=model_path)
                param = [COUNT, _drop, _lr, _batch]
                param.extend(scores)
                score_list.append(param)
                COUNT = COUNT + 1

    # ******************* validation balances frames  ********************
    
    print(score_list)
    
    df_res = pd.DataFrame(score_list, columns = ["id", "drop", "lr", "batch", "score", "f1", "acc"])
    display(df_res)
    file_out = dir_out + "\\res0_" + model_path_base + ".csv"
    df_res.to_csv(file_out, index=False)
    
    best_id = df_res.loc[:,"score"].idxmax()
    
    best_path = dir_out + "\\" + model_path_base + "_{0:02d}.h5".format(best_id)
    out_path = dir_out + "\\" + model_path_base + "_best.h5"
    #best_path = dir_out + "\\model_image_single_b2_pseudo" + "_{0:02d}.h5".format(best_id)
    #out_path = dir_out + "\\model_image_single_b2_pseudo_best.h5"
    shutil.copy(best_path, out_path)

    df_train_list = None
    df_train = None
    df_val = None
    df_val_list = None
    np_train_x = None
    np_train_y = None
    np_val_x = None
    np_val_y = None
    
    # ******************* validation all frames  ********************
    
    #file_model = dir_out + "\\model_image_single_pseudo_best.h5"
    #file_model = dir_out + "\\model_image_single_best.h5"
    model = keras.models.load_model(out_path, custom_objects={'f1':f1})
    
    model.summary()
    
    #val_list = mFILE.search_files(dir_label_val, valid_names=[".txt"], invalid_names=["wuert"], ext=None, recursive=False)
    #len_val = len(val_list)
    
    # ******* not interpolate ********
    df_val_list = []
    for i in tqdm(range(len_val)):
        name = os.path.splitext(os.path.basename(val_list[i]))[0]
        
        df_label = pd.read_csv(val_list[i])
        df_label = df_label.drop(["Anger","Disgust","Fear","Happiness","Sadness","Surprise"],axis=1)
        df_label.columns = ["expr"]
        df_label["frame"] = df_label.index
        
        df_data = pd.read_hdf(dir_data + "\\" + name + ".h5")
        
        df_merge = pd.merge(df_label, df_data, on="frame", how='outer')
        # interpolate 20 frame back
        #df_merge.interpolate(method="index", limit=20, limit_direction='backward', inplace=True)
        df_merge = df_merge.loc[df_merge.loc[:,"expr"]>=0,:]
                
        df_val_list.append(df_merge)
    
    df_val = pd.concat([x for x in df_val_list],axis=0)
    df_val_list = None #clear memory ***************
    df_val.fillna(0, inplace=True)
    display(df_val)
    
    np_val_x = df_val.loc[:, df_val.columns.str.contains("AU|pose_R|gaze|vgg-")].values
    np_val_y = df_val.loc[:, df_val.columns.str.contains("expr")].astype("int").values #.ravel()
    
    display(np_val_x.shape, np_val_y.shape)
       
    dummy = [[0],[1],[2],[3],[4],[5],[6]]
    transformer = OneHotEncoder().fit(dummy)
    np_val_y_hot = transformer.transform(np_val_y).toarray()
    
    pred_nn = model.predict(np_val_x)
    score_nn_f = f1_score(np.argmax(np_val_y_hot, axis=1), np.argmax(pred_nn, axis=1), average='macro')
    score_nn_a = accuracy_score(np.argmax(np_val_y_hot, axis=1), np.argmax(pred_nn, axis=1))
    score_nn = score_nn_f*0.67 + score_nn_a*0.33
    
    scores_nn = [score_nn, score_nn_f, score_nn_a]
    display("w/o interpolate")
    display(["0.67*F1+0.33*ACC", "F1 score", "ACC score"])
    display(scores_nn)
    
    df_out = eval_pred_each_class(np.argmax(np_val_y_hot, axis=1), np.argmax(pred_nn, axis=1), 7)
    display(df_out)
    
    file_out = dir_out + "\\res1_" + model_path_base + ".csv"
    df_out.to_csv(file_out, index=False)
    
    # ******* with interpolate ********
    df_val_list = []
    for i in tqdm(range(len_val)):
        name = os.path.splitext(os.path.basename(val_list[i]))[0]
        
        df_label = pd.read_csv(val_list[i])
        df_label = df_label.drop(["Anger","Disgust","Fear","Happiness","Sadness","Surprise"],axis=1)
        df_label.columns = ["expr"]
        df_label["frame"] = df_label.index
        
        df_data = pd.read_hdf(dir_data + "\\" + name + ".h5")
        
        df_merge = pd.merge(df_label, df_data, on="frame", how='outer')
        # interpolate 30 frame back
        df_merge.interpolate(method="index", limit=30, limit_direction='backward', inplace=True)
        df_merge = df_merge.loc[df_merge.loc[:,"expr"]>=0,:]
                
        df_val_list.append(df_merge)
    
    df_val = pd.concat([x for x in df_val_list],axis=0)
    df_val_list = None #clear memory ***************
    df_val.fillna(0, inplace=True)
    display(df_val)
    
    np_val_x = df_val.loc[:, df_val.columns.str.contains("AU|pose_R|gaze|vgg-")].values
    np_val_y = df_val.loc[:, df_val.columns.str.contains("expr")].astype("int").values #.ravel()
    
    display(np_val_x.shape, np_val_y.shape)
       
    dummy = [[0],[1],[2],[3],[4],[5],[6]]
    transformer = OneHotEncoder().fit(dummy)
    np_val_y_hot = transformer.transform(np_val_y).toarray()
    
    pred_nn = model.predict(np_val_x)
    score_nn_f = f1_score(np.argmax(np_val_y_hot, axis=1), np.argmax(pred_nn, axis=1), average='macro')
    score_nn_a = accuracy_score(np.argmax(np_val_y_hot, axis=1), np.argmax(pred_nn, axis=1))
    score_nn = score_nn_f*0.67 + score_nn_a*0.33
    
    scores_nn = [score_nn, score_nn_f, score_nn_a]
    display("with interpolate")
    display(["0.67*F1+0.33*ACC", "F1 score", "ACC score"])
    display(scores_nn)
    
    df_out = eval_pred_each_class(np.argmax(np_val_y_hot, axis=1), np.argmax(pred_nn, axis=1), 7)
    display(df_out)
    
    file_out = dir_out + "\\res2_" + model_path_base + ".csv"
    df_out.to_csv(file_out, index=False)
    

In [ ]:
if __name__ == "__main__":
    main()